# The data

The data is a list of article metadata from NYT for the month of January through
the years 2013 to 2024. The data is in JSON format. 

Our goal is to import all the saved data and create a pandas DataFrame with it.
This will let us analyze the data and answer questions like:

- Trends in article topics over the last 10 years
- Most popular authors
- Most popular sections
- Most popular keywords
- Most popular articles
- and so on...

In [ ]:
import json
import requests
import time
import os
from dotenv import dotenv_values

config = dotenv_values(".env")

API_KEY = config['API_KEY']

def get_nyt_articles(year, month):
    url = f'https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={API_KEY}'
    response = requests.get(url)
    response.raise_for_status()
    # return only the articles. The response object contains metadata as well.
    return response.json()['response']['docs']

def build_nyt_archive():
    articles = []
    month = 1
    for year in range(2014, 2025):
        articles.extend(get_nyt_articles(year, month))
        print(f'Fetched {len(articles)} articles total.')
        time.sleep(20)
    return articles

def save_nyt_archive(articles):
    # check if the data folder exists, if not, create it
    if not os.path.exists('data'):
        os.makedirs('data')
    with open('data/nyt_archive_2014_2024_jan.json', 'w') as f:
        json.dump(articles, f)

articles = build_nyt_archive()
save_nyt_archive(articles)
print('NYT archive saved to nyt_archive_2013_2023_jan.json')



In [2]:
def load_json():
    with open('data/nyt_archive_2014_2024_jan.json', 'r') as f:
        return json.load(f)
    
articles = load_json()
print(len(articles))
print(articles[0])

57364
{'abstract': 'Other than the national championship game, the Rose Bowl offers the most anticipated matchup of the season, but the Capital One Bowl and Fiesta Bowl should also be intriguing.', 'web_url': 'https://www.nytimes.com/2014/01/01/sports/ncaafootball/new-years-day-bowl-games.html', 'snippet': 'Other than the national championship game, the Rose Bowl offers the most anticipated matchup of the season, but the Capital One Bowl and Fiesta Bowl should also be intriguing.', 'lead_paragraph': 'ROSE BOWL', 'print_section': 'B', 'print_page': '10', 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2014/01/01/sports/Y-games/Y-games-articleLarge.jpg', 'height': 318, 'width': 600, 'subType': 'xlarge', 'crop_name': 'articleLarge', 'legacy': {'xlarge': 'images/2014/01/01/sports/Y-games/Y-games-articleLarge.jpg', 'xlargewidth': 600, 'xlargeheight': 318}}, {'rank': 0, 'subtype': 'jumbo', 'capti

In [4]:
# Exploration

# Each article is a dictionary with multiple keys. Some of the values are
# dictionaries themselves. For example, the 'headline' key has a dictionary
# as its value. The 'keywords' key has a list of dictionaries as its value.

# keys of the articles dictionary
print(articles[0].keys())

# keys of the headline dictionary
print(articles[0]['headline'].keys())

# keys of the first keyword dictionary
print(articles[0]['keywords'][0].keys())

# multimedia is a list of dictionaries for the multimedia content of the article
# count of the multimedia content of the first article
print(len(articles[0]['multimedia']))

# the data I think we should keep from the articles dictionary is:
# abstract, byline (Author Name), document_type, headline, keywords, news_desk, section_name, word_count, 

# print the data in the keywords field
print(articles[0]['keywords'])

# print the data in the headline field
print(articles[0]['headline'])

# print the data in the byline field
print(articles[0]['byline'])



dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'subsection_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])
dict_keys(['main', 'kicker', 'content_kicker', 'print_headline', 'name', 'seo', 'sub'])
dict_keys(['name', 'value', 'rank', 'major'])
5
[{'name': 'persons', 'value': 'Bierman, Fred', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Rose Bowl (Football Game)', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Football (College)', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Stanford University', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Michigan State University', 'rank': 5, 'major': 'N'}, {'name': 'subject', 'value': 'Fiesta Bowl', 'rank': 6, 'major': 'N'}, {'name': 'organizations', 'value': 'University of South Carolina', 'rank': 7, 'major': 'N'}, {'name': 'organ

# Creating dataframes

We will create a pandas DataFrame from the data based on the fields we've
decided to keep.

In [32]:
import pandas as pd
import numpy as np


def extract_keywords_with_subcategories(article):
    """Extracts keywords and keeps the first rank as main and second rank as subcategory."""
    keyword_dict = {}
    
    # Initialize lists to hold keywords by type
    subjects = []
    organizations = []
    glocations = []
    persons = []

    # Iterate over keywords and categorize them
    for keyword in article.get('keywords', []):
        keyword_type = keyword['name']
        keyword_value = keyword['value']
        
        # Append the keyword to the appropriate list
        if keyword_type == 'subject':
            subjects.append(keyword_value)
        elif keyword_type == 'organizations':
            organizations.append(keyword_value)
        elif keyword_type == 'glocations':
            glocations.append(keyword_value)
        elif keyword_type == 'persons':
            keyword_dict['person'] = keyword_value

    # Assign the first and second ranked keywords for each type
    if subjects:
        keyword_dict['subject'] = subjects[0]  # First ranked subject
        keyword_dict['subject_subcategory'] = subjects[1] if len(subjects) > 1 else np.nan  # Second ranked subject

    if organizations:
        keyword_dict['organization'] = organizations[0]  # First ranked organization
        keyword_dict['organization_subcategory'] = organizations[1] if len(organizations) > 1 else np.nan  # Second ranked organization

    if glocations:
        keyword_dict['glocation'] = glocations[0]  # First ranked glocation
        keyword_dict['glocation_subcategory'] = glocations[1] if len(glocations) > 1 else np.nan  # Second ranked glocation

    if persons:
        keyword_dict['person'] = persons[0]
        keyword_dict['person_subcategory'] = persons[1] if len(persons) > 1 else np.nan

    return keyword_dict

# Data processing loop
articles_data = []
for article in articles:
    article_data = {
        'headline': article['headline']['main'],
        'pub_date': article['pub_date'],
        'document_type': article['document_type'],
        'word_count': article.get('word_count', 0),
        'news_desk': article.get('news_desk'),
        'section_name': article.get('section_name'),
        'type_of_material': article.get('type_of_material'),
        'multimedia_count': len(article.get('multimedia', [])),
        'byline': article.get('byline', {}).get('original', np.nan),
    }
    # Add the keywords with subcategories
    article_data.update(extract_keywords_with_subcategories(article))

    articles_data.append(article_data)

# Create DataFrame
df = pd.DataFrame(articles_data)

df.to_csv('data/raw.csv', index=False)

In [33]:
# count the null values in each row and add a new column to the dataframe for
# the count of null values.
df['null_count'] = df.isnull().sum(axis=1)

# sort the dataframe by the number of null values in each row
df = df.sort_values('null_count', ascending=True)

# convert the all entries in the whole dataframe to title case
df = df.apply(lambda x: x.str.title() if x.dtype == "object" else x)

# save the cleaned data to a new csv file
df.to_csv('data/cleaned.csv', index=False)

# Cleaning Data

For this next part we are going clean some of the data columns. Everything
after multimedia counts is a keyword. We need to make sure we are able to
separate them into their own columns so that it's a little clearer to see what
is going on. Right now the names of the keywords are the column headers and
the values are in the rows. 

In [20]:
# print the columns of the dataframe
#print(df.columns)

# let's find out unique values in the subject column and the count.
# print(df['subject'].value_counts())

# right now, the subject column groups each of the subjects into a single
# string, separated by commas. Let's split them into separate columns. We will
# name them subject_1, subject_2, subject_3, and so on. This will depend on
# the number of subjects for each article. Some have only one subject, while
# others have multiple subjects.

# print(df['subject'].head())
# for i in range(20):
#     print(df.iloc[i]['subject'])
# print(df.iloc[0]['subject'])
# get the maximum number of subjects for an article
# max_subjects = df['subject'].str.split(', ')

# split_data = max_subjects.dropna().apply(lambda x: [item.strip() for item in x[0].split(',')])
# print(split_data)

# max_subjects = split_data.apply(len).max()


# print(max_subjects)

# print(type(max_subjects))
# # create a new dataframe with the subject columns
# df_subjects = df['subject'].str.split(',', expand=True)

# # rename the columns
# df_subjects.columns = [f'subject_{i+1}' for i in range(max_subjects)]

# # combine the two dataframes
# df = pd.concat([df, df_subjects], axis=1)

# # drop the original subject column
# df = df.drop('subject', axis=1)

# print(df.head())

df.loc[df.glocations.notna(), 'glocations'] = df.glocations.str.title()


# get the count of subjects for each article
df['subject_count'] = df['subject'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)

# get the row with the maximum number of subjects
max_subject_row = df['subject_count'].idxmax()

# count for the maximum number of subjects
# print(df['subject_count'].max())

# print the row with the maximum number of subjects

# print(df['subject_count'].head())

subjects_split = df['subject'].str.split(',', expand=True)

subjects_split.columns = [f"subject_{i+1}" for i in range(subjects_split.shape[1])]

# df_with_subjects.to_csv('data/nyt_archive_2014_2024_jan.csv', index=False)

# We will have to do the same thing with the organizations and glocations
# columns.

# get the count of organizations for each article
df['organizations_count'] = df['organizations'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)

# get the row with the maximum number of organizations
max_organizations_row = df['organizations_count'].idxmax()



organizations_split = df['organizations'].str.split(',', expand=True)

organizations_split.columns = [f"organization_{i+1}" for i in range(organizations_split.shape[1])]
# df_with_organizations = pd.concat([df, organizations_split], axis=1)

# get the count of glocations for each article
df['glocations_count'] = df['glocations'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)

# get the row with the maximum number of glocations
max_glocations_row = df['glocations_count'].idxmax()



glocations_split = df['glocations'].str.split(',', expand=True)

glocations_split.columns = [f"glocation_{i+1}" for i in range(glocations_split.shape[1])]


df_separated = pd.concat([df, subjects_split, organizations_split, glocations_split], axis=1)

df_separated.to_csv('data/nyt_archive_2014_2024_jan.csv', index=False)




# Date cleaning

The date column is a string. We can use regex to extract the year, month, and
day from the date and put them in their own columns.

In [21]:
# date cleaning
# convert the pub_date column to a datetime object
df_separated['pub_date'] = pd.to_datetime(df_separated['pub_date'])

# get the year, month, and day from the pub_date column
df_separated['year'] = df_separated['pub_date'].dt.year
df_separated['month'] = df_separated['pub_date'].dt.month
df_separated['day'] = df_separated['pub_date'].dt.day

# drop the pub_date column
df_separated = df_separated.drop('pub_date', axis=1)

# save the cleaned data to a new CSV file
df_separated.to_csv('data/nyt_archive_2014_2024_jan_cleaned.csv', index=False)

# Author cleaning

The author column is a string. We can use regex to extract the author name
from the author column and put it in its own column. 

In [13]:
# author cleaning

# get the unique values in the byline column
print(df_separated['byline'].value_counts())

                                            6278
The New York Times                          1934
By The New York Times                        708
By The Editorial Board                       531
By The Learning Network                      446
                                            ... 
By Lauren Hard                                 1
By Jeremy D. Goodwin                           1
By Robert W. Goldfarb                          1
By Emily Cochrane and Michael S. Schmidt       1
By Claire Moses and Orlando Mayorquin          1
Name: byline, Length: 10760, dtype: int64


In [14]:
# remove the 'By ' prefix from the byline column
df_separated['byline'] = df_separated['byline'].str.replace('By ', '')

# get unique values in the byline column
print(df_separated['byline'].value_counts())

                                      6278
The New York Times                    2642
The Editorial Board                    531
The Learning Network                   446
Paul Krugman                           306
                                      ... 
Maryanne Garbowsky                       1
Niraj Chokshi and Daniel Victor          1
Jordan Rau                               1
Amir Ahmadi Arian                        1
Claire Moses and Orlando Mayorquin       1
Name: byline, Length: 10709, dtype: int64


In [15]:
# rename byline column to author
df_separated = df_separated.rename(columns={'byline': 'author'})

# save the cleaned data to a new CSV file
df_separated.to_csv('data/nyt_archive_2014_2024_jan_cleaned.csv', index=False)


In [16]:
# some articles have multiple authors and some have none. Some have only one
# author. The multiple authors are split by commas and the word 'and'. We can
# split the authors into separate columns. We will name them author_1,
# author_2,
# author_3, and so on. This will depend on the number of authors for each 
# article. 

# we can do this similar to how we split the subjects, organizations, and
# glocations columns.

# get the count of authors for each article
df_separated['author_count'] = df_separated['author'].apply(lambda x: len(x.split(', | and')) if pd.notna(x) else 0)

# get the row with the maximum number of authors
max_author_row = df_separated['author_count'].idxmax()

# split the authors into separate columns
authors_split = df_separated['author'].str.split(', | and', expand=True)

authors_split.columns = [f"author_{i+1}" for i in range(authors_split.shape[1])]

df_separated = pd.concat([df_separated, authors_split], axis=1)

# save the cleaned data to a new CSV file
df_separated.to_csv('data/nyt_archive_2014_2024_jan_cleaned.csv', index=False)

In [26]:
df_separated['null_count'] = df_separated.isnull().sum(axis=1)

df_sorted = df_separated.sort_values('null_count', ascending=True)

print(df_sorted.head()) 

df_less = df_sorted[:15000]

df_less.to_csv('data/nyt_archive_clean.csv', index=False)

                                                headline document_type  \
54440                            52 Places to Go in 2024    multimedia   
51354                            52 Places to Go in 2023    multimedia   
35824                            52 Places to Go in 2020    multimedia   
47321                      52 Places for a Changed World    multimedia   
37168  You Don’t Need to Travel to Profit From Intern...       article   

       word_count       news_desk  section_name     type_of_material  \
54440           0          Travel        Travel  Interactive Feature   
51354           0          Travel        Travel  Interactive Feature   
35824           0          Travel        Travel  Interactive Feature   
47321           0          Travel        Travel  Interactive Feature   
37168         990  SundayBusiness  Business Day                 News   

       multimedia_count                 byline persons  \
54440                 5  By The New York Times     NaN   
51354 